# Instituto Res Publica

## Extracción de links

In [3]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
from selenium.webdriver.chrome.options import Options

def create_driver(headless=True):
    """Crea y retorna un WebDriver con la opción de ser headless si se desea."""
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def extract_articles_from_url2(driver, url, existing_df=None):
    """
    Extrae información de artículos desde una URL y agrega los resultados a un DataFrame existente.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): URL de la página para extraer artículos.
        existing_df (pd.DataFrame, optional): DataFrame al que agregar los artículos extraídos. Si no se proporciona, se crea uno nuevo.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos agregados al existente (o uno nuevo si no se proporciona).
    """
    try:
        driver.get(url)
        # Esperar a que cargue el contenido
        time.sleep(3)

        # Obtener todos los artículos dentro de las etiquetas <article>
        articles = driver.find_elements(By.CSS_SELECTOR, "article.et_pb_post")

        data = []
        for article in articles:
            # Imagen
            try:
                img = article.find_element(By.CSS_SELECTOR, "a.entry-featured-image-url img")
                image_url = img.get_attribute("src")
            except:
                image_url = None

            # Título y enlace
            try:
                title_element = article.find_element(By.CSS_SELECTOR, "h2.entry-title a")
                title = title_element.text.strip()
                link = title_element.get_attribute("href")
            except:
                continue  # Si no hay título o enlace, saltamos este artículo

            # Fecha
            try:
                date = article.find_element(By.CSS_SELECTOR, "span.published").text.strip()
            except:
                date = "No especificada"

            # Contenido (puedes agregar más lógica si el contenido está en una clase específica)
            try:
                content_element = article.find_element(By.CSS_SELECTOR, "div.post-content-inner")
                content = content_element.text.strip()
            except:
                content = None

            # Crear diccionario con los datos
            article_data = {
                'Título': title,
                'Enlace': link,
                'Fecha Publicación': date,
                'URL Imagen': image_url,
                'Contenido': content,
                'Fecha Consulta': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            data.append(article_data)

        # Crear DataFrame con los nuevos datos
        new_df = pd.DataFrame(data)

        # Si ya existe un DataFrame proporcionado, concatenar los nuevos datos al existente
        if existing_df is not None:
            existing_df = pd.concat([existing_df, new_df], ignore_index=True)
        else:
            # Si no existe, devolver el nuevo DataFrame
            existing_df = new_df

        return existing_df

    except Exception as e:
        print(f"Error en la extracción: {str(e)}")
        return existing_df if existing_df is not None else pd.DataFrame()

def extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=100, headless=True):
    """
    Itera sobre varias páginas y extrae los artículos, deteniéndose cuando una página no se encuentra.
    
    Args:
        base_url (str): URL base de las páginas, debe contener un marcador de página, e.g., 'page/{page_number}/'.
        start_page (int): Número de página inicial para empezar a iterar (por defecto es la página 1).
        max_pages (int): Número máximo de páginas a intentar antes de detener la búsqueda (por defecto es 100).
        headless (bool): Si se debe ejecutar el navegador en modo headless (sin abrir ventana). Por defecto es True.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos de todas las páginas procesadas.
    """
    driver = create_driver(headless)  # Inicializar el WebDriver solo una vez
    current_page = start_page
    all_articles_df = pd.DataFrame()  # Inicializamos un DataFrame vacío

    while current_page <= max_pages:
        # Construir la URL de la página actual
        url = f"{base_url}{current_page}/"  # Aquí el formato de la URL, el número de página se añade al final
        print(f"Extrayendo artículos de: {url}")

        # Llamar a la función extract_articles_from_url2 para obtener artículos de la página
        all_articles_df = extract_articles_from_url2(driver, url, all_articles_df)

        # Verificar si se encontró una página de error (por ejemplo, página no encontrada)
        if all_articles_df is None or len(all_articles_df) == 0:
            print(f"No se encontraron artículos en la página {current_page}, o la página no existe.")
            break

        current_page += 1  # Pasar a la siguiente página

    driver.quit()  # Cerrar el WebDriver después de todas las iteraciones
    return all_articles_df

In [6]:
def extract_articles_from_url2(driver, url, existing_df=None):
    """
    Extrae información de artículos desde una URL y agrega los resultados a un DataFrame existente.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): URL de la página para extraer artículos.
        existing_df (pd.DataFrame, optional): DataFrame al que agregar los artículos extraídos. Si no se proporciona, se crea uno nuevo.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos agregados al existente (o uno nuevo si no se proporciona).
    """
    try:
        driver.get(url)
        # Esperar a que cargue el contenido
        time.sleep(3)

        # Obtener todos los artículos dentro de las secciones
        categories = driver.find_elements(By.CSS_SELECTOR, "div.row.editorial-category")

        data = []
        for category in categories:
            # Obtener los enlaces dentro de la sección de artículos
            section_items = category.find_elements(By.CSS_SELECTOR, "div.section-items a")

            for item in section_items:
                # Obtener el enlace (URL) del artículo
                link = item.get_attribute("href")

                # Obtener el título del artículo (dentro de <h4>)
                try:
                    title = item.find_element(By.CSS_SELECTOR, "h4").text.strip()
                except:
                    title = None  # Si no hay título, asignamos None

                # Obtener la URL de la imagen (dentro de <img>)
                try:
                    img = item.find_element(By.CSS_SELECTOR, "img")
                    image_url = img.get_attribute("src")
                except:
                    image_url = None  # Si no hay imagen, asignamos None

                # Extraer la fecha (no está explícita en esta estructura, por lo que podemos dejarla como "No especificada")
                date = "No especificada"

                # Crear diccionario con los datos
                article_data = {
                    'Título': title,
                    'Enlace': link,
                    'Fecha Publicación': date,
                    'URL Imagen': image_url,
                    'Contenido': None,  # En esta estructura no parece haber un contenido textual
                    'Fecha Consulta': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                }

                data.append(article_data)

        # Crear DataFrame con los nuevos datos
        new_df = pd.DataFrame(data)

        # Si ya existe un DataFrame proporcionado, concatenar los nuevos datos al existente
        if existing_df is not None:
            existing_df = pd.concat([existing_df, new_df], ignore_index=True)
        else:
            # Si no existe, devolver el nuevo DataFrame
            existing_df = new_df

        return existing_df

    except Exception as e:
        print(f"Error en la extracción: {str(e)}")
        return existing_df if existing_df is not None else pd.DataFrame()

Editorial

In [8]:
base_url = "https://www.respublica.cl/sections/editorial/"
edit = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=1, headless=False)

Extrayendo artículos de: https://www.respublica.cl/sections/editorial/1/


In [9]:
edit

,Título,Enlace,Fecha Publicación,URL Imagen,Contenido,Fecha Consulta
0,Aborto y eutanasia. Dilemas sobre el derecho a...,https://www.tiendarespublica.cl/aborto-y-eutan...,No especificada,https://www.respublica.cl/img/uploads/7136ed3a...,None,2024-11-29 09:58:58
1,El anillo de Giges,https://www.tiendarespublica.cl/el-anillo-de-g...,No especificada,https://www.respublica.cl/img/uploads/1b9209a7...,None,2024-11-29 09:58:58
2,Ética de la Libertad,https://www.tiendarespublica.cl/etica-de-la-li...,No especificada,https://www.respublica.cl/img/uploads/a52d4dda...,None,2024-11-29 09:58:58
3,¿Para qué sirve la política?,https://www.tiendarespublica.cl/para-que-sirve...,No especificada,https://www.respublica.cl/img/uploads/6210df4f...,None,2024-11-29 09:58:58
4,Breve historia del pensamiento político modern...,https://www.tiendarespublica.cl/para-que-sirve...,No especificada,https://www.respublica.cl/img/uploads/bfa67b5a...,None,2024-11-29 09:58:59
5,Subsidiariedad en Chile,https://www.tiendarespublica.cl/subsidiariedad...,No especificada,https://www.respublica.cl/img/uploads/e45e1e26...,None,2024-11-29 09:58:59
6,Raíces Cristianas de la Economía de Libre Mercado,https://www.tiendarespublica.cl/raices-cristia...,No especificada,https://www.respublica.cl/img/uploads/6ffece54...,None,2024-11-29 09:58:59
7,"Historia, Ideas y Política",https://www.tiendarespublica.cl/historia-ideas...,No especificada,https://www.respublica.cl/img/uploads/c62e7eb7...,None,2024-11-29 09:58:59
8,En la víspera de una nueva catástrofe europea:...,https://www.tiendarespublica.cl/en-la-vispera-...,No especificada,https://www.respublica.cl/img/uploads/ceec70c5...,None,2024-11-29 09:58:59
9,¿Qué es El Matrimonio? hombre y mujer: una def...,https://www.tiendarespublica.cl/que-es-matrimo...,No especificada,https://www.respublica.cl/img/uploads/040c6fdb...,None,2024-11-29 09:58:59


Ensayos y articulos

In [11]:
def extract_articles_from_url2(driver, url, existing_df=None):
    """
    Extrae información de artículos desde una URL y agrega los resultados a un DataFrame existente.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): URL de la página para extraer artículos.
        existing_df (pd.DataFrame, optional): DataFrame al que agregar los artículos extraídos. Si no se proporciona, se crea uno nuevo.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos agregados al existente (o uno nuevo si no se proporciona).
    """
    try:
        driver.get(url)
        # Esperar a que cargue el contenido
        time.sleep(3)

        # Obtener todos los elementos dentro de .section-items
        section_items = driver.find_elements(By.CSS_SELECTOR, "div.section-items .section-item")

        data = []
        for item in section_items:
            # Título y enlace
            try:
                link_element = item.find_element(By.CSS_SELECTOR, "a")
                link = link_element.get_attribute("href")
                title = item.find_element(By.CSS_SELECTOR, "h2").text.strip()
            except:
                continue  # Si no hay título o enlace, saltamos este artículo

            # Imagen
            try:
                img_style = item.find_element(By.CSS_SELECTOR, "div.item-img img").get_attribute("style")
                # Extraer la URL de la imagen desde el atributo 'style' (background-image:url(...))
                image_url = img_style.split('url(')[1].split(')')[0].strip('"')
            except:
                image_url = None

            # Crear diccionario con los datos
            article_data = {
                'Título': title,
                'Enlace': link,
                'URL Imagen': image_url,
                'Fecha Consulta': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            data.append(article_data)

        # Crear DataFrame con los nuevos datos
        new_df = pd.DataFrame(data)

        # Si ya existe un DataFrame proporcionado, concatenar los nuevos datos al existente
        if existing_df is not None:
            existing_df = pd.concat([existing_df, new_df], ignore_index=True)
        else:
            # Si no existe, devolver el nuevo DataFrame
            existing_df = new_df

        return existing_df

    except Exception as e:
        print(f"Error en la extracción: {str(e)}")
        return existing_df if existing_df is not None else pd.DataFrame()

In [12]:
base_url = "https://www.respublica.cl/sections/ensayos-y-articulos/"
eya = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=1, headless=False)
eya

Extrayendo artículos de: https://www.respublica.cl/sections/ensayos-y-articulos/1/


,Título,Enlace,URL Imagen,Fecha Consulta
0,OBSERVATORIO INTERNACIONAL,https://www.respublica.cl/ss-ensayos-y-articul...,https://www.respublica.cl/img/uploads/5f29c763...,2024-11-29 10:03:34
1,MIRADA PÚBLICA,https://www.respublica.cl/ss-ensayos-y-articul...,https://www.respublica.cl/img/uploads/7f7a09b5...,2024-11-29 10:03:34
2,ENTREVISTAS PÚBLICAS,https://www.respublica.cl/ss-ensayos-y-articul...,https://www.respublica.cl/img/uploads/380bc5eb...,2024-11-29 10:03:34
3,ANÁLISIS DE ACTUALIDAD,https://www.respublica.cl/ss-ensayos-y-articul...,https://www.respublica.cl/img/uploads/b98b6900...,2024-11-29 10:03:34
4,PERSONAJES PÚBLICOS,https://www.respublica.cl/ss-ensayos-y-articul...,https://www.respublica.cl/img/uploads/60ab252e...,2024-11-29 10:03:34
5,PENSADORES FUNDAMENTALES,https://www.respublica.cl/ss-ensayos-y-articul...,https://www.respublica.cl/img/uploads/0433cf98...,2024-11-29 10:03:34
6,DISCURSO PÚBLICO,https://www.respublica.cl/ss-ensayos-y-articul...,https://www.respublica.cl/img/uploads/cadfba02...,2024-11-29 10:03:35
7,MIRADA PÚBLICA: ESPECIAL DEBATE CONSTITUCIONAL,https://www.respublica.cl/ss-ensayos-y-articul...,https://www.respublica.cl/img/uploads/ee1f91b6...,2024-11-29 10:03:35
8,PRINCIPIOS CONSTITUCIONALES,https://www.respublica.cl/ss-ensayos-y-articul...,https://www.respublica.cl/img/uploads/47032cac...,2024-11-29 10:03:35
9,DOCUMENTOS FUNDAMENTALES,https://www.respublica.cl/ss-ensayos-y-articul...,https://www.respublica.cl/img/uploads/6140686d...,2024-11-29 10:03:35


Estudios

In [13]:
base_url = "https://www.respublica.cl/sections/estudios/"
est = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=1, headless=False)
est

Extrayendo artículos de: https://www.respublica.cl/sections/estudios/1/


,Título,Enlace,URL Imagen,Fecha Consulta
0,Nuevo escenario político de gobernadores 2024,https://www.respublica.cl/estudio/1152,https://www.respublica.cl/img/uploads/893df9a8...,2024-11-29 10:04:34
1,Informe Elecciones CORE y Consejos Municipales...,https://www.respublica.cl/estudio/1144,https://www.respublica.cl/img/uploads/cbef0aca...,2024-11-29 10:04:34
2,PRESENTACIÓN: RADIOGRAFÍA AL SENADO 2023,https://www.respublica.cl/estudio/1024,https://www.respublica.cl/img/uploads/6c051671...,2024-11-29 10:04:34
3,Estudio| Radiografía al Senado 2023,https://www.respublica.cl/estudio/1025,https://www.respublica.cl/img/uploads/83eec251...,2024-11-29 10:04:34
4,Estudio|Radiografía a la Cámara de Diputados 2023,https://www.respublica.cl/estudio/1051,https://www.respublica.cl/img/uploads/090e8c11...,2024-11-29 10:04:34
5,Ranking de Servicios de Salud en Chile Revela ...,https://www.respublica.cl/estudio/1107,https://www.respublica.cl/img/uploads/c40afc78...,2024-11-29 10:04:34
6,ANÁLISIS A LA REFORMA DE PENSIONES,https://www.respublica.cl/estudio/1119,https://www.respublica.cl/img/uploads/294f94f8...,2024-11-29 10:04:34
7,"RADIOGRAFÍA A LA COMISIÓN EXPERTA, REDACTORA D...",https://www.respublica.cl/estudio/889,https://www.respublica.cl/img/uploads/314e7b56...,2024-11-29 10:04:35
8,RADIOGRAFÍA A LA CÁMARA DE DIPUTADOS DE LA NAC...,https://www.respublica.cl/estudio/993,https://www.respublica.cl/img/uploads/a0ed0cf5...,2024-11-29 10:04:35
9,INFORMACIÓN DEL PLENO DEL CONSEJO CONSTITUCION...,https://www.respublica.cl/estudio/994,https://www.respublica.cl/img/uploads/baca084d...,2024-11-29 10:04:35


Noticias

In [15]:
from datetime import datetime
import time
import pandas as pd
from selenium.webdriver.common.by import By

def extract_articles_from_url2(driver, url, existing_df=None):
    """
    Extrae información de artículos desde una URL y agrega los resultados a un DataFrame existente.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): URL de la página para extraer artículos.
        existing_df (pd.DataFrame, optional): DataFrame al que agregar los artículos extraídos. Si no se proporciona, se crea uno nuevo.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos agregados al existente (o uno nuevo si no se proporciona).
    """
    try:
        driver.get(url)
        # Esperar a que cargue el contenido
        time.sleep(3)

        # Obtener todos los elementos dentro de .section-items
        section_items = driver.find_elements(By.CSS_SELECTOR, "div.section-items .section-item")

        data = []
        for item in section_items:
            # Título y enlace
            try:
                link_element = item.find_element(By.CSS_SELECTOR, "a")
                link = link_element.get_attribute("href")
                title = item.find_element(By.CSS_SELECTOR, "h2").text.strip()
            except Exception as e:
                print(f"Error al obtener el título o enlace: {e}")
                continue  # Si no hay título o enlace, saltamos este artículo

            # Imagen
            try:
                img_style = item.find_element(By.CSS_SELECTOR, "div.item-img img").get_attribute("style")
                # Extraer la URL de la imagen desde el atributo 'style' (background-image:url(...))
                image_url = img_style.split('url(')[1].split(')')[0].strip('"')
            except Exception as e:
                image_url = None
                print(f"Error al obtener la imagen: {e}")

            # Fecha
            try:
                date = item.find_element(By.CSS_SELECTOR, "p").text.strip()
            except Exception as e:
                date = None
                print(f"Error al obtener la fecha: {e}")

            # Crear diccionario con los datos
            article_data = {
                'Título': title,
                'Enlace': link,
                'URL Imagen': image_url,
                'Fecha': date,
                'Fecha Consulta': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            data.append(article_data)

        # Crear DataFrame con los nuevos datos
        new_df = pd.DataFrame(data)

        # Si ya existe un DataFrame proporcionado, concatenar los nuevos datos al existente
        if existing_df is not None:
            existing_df = pd.concat([existing_df, new_df], ignore_index=True)
        else:
            # Si no existe, devolver el nuevo DataFrame
            existing_df = new_df

        return existing_df

    except Exception as e:
        print(f"Error en la extracción: {str(e)}")
        return existing_df if existing_df is not None else pd.DataFrame()

In [16]:
base_url = "https://www.respublica.cl/sections/noticias/"
notic = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=1, headless=False)
notic

Extrayendo artículos de: https://www.respublica.cl/sections/noticias/1/


,Título,Enlace,URL Imagen,Fecha,Fecha Consulta
0,ALERTA TEMPRANA N°46: NUEVO PARO DE FUNCIONARI...,https://www.respublica.cl/articulo/1155,https://www.respublica.cl/img/uploads/36183741...,"28 de Noviembre, 2024",2024-11-29 10:09:46
1,ALERTA TEMPRANA N°45 NUEVO ESCENARIO POLÍTICO ...,https://www.respublica.cl/articulo/1151,https://www.respublica.cl/img/uploads/0d6e962e...,"25 de Noviembre, 2024",2024-11-29 10:09:46
2,ALERTA TEMPRANA N°44 USO DE RECURSOS FISCALES ...,https://www.respublica.cl/articulo/1149,https://www.respublica.cl/img/uploads/3ec61926...,"21 de Noviembre, 2024",2024-11-29 10:09:46
3,ALERTA TEMPRANA N°43 LAS INCOHERENCIAS DEL GOB...,https://www.respublica.cl/articulo/1150,https://www.respublica.cl/img/uploads/8f6b03c8...,"21 de Noviembre, 2024",2024-11-29 10:09:47
4,ALERTA TEMPRANA N°42 ¿ES LEGAL UN PARO DE FUNC...,https://www.respublica.cl/articulo/1146,https://www.respublica.cl/img/uploads/f510a91f...,"19 de Noviembre, 2024",2024-11-29 10:09:47
...,...,...,...,...,...
185,Nuestro Director de Contenidos Alvaro Iriarte ...,https://www.respublica.cl/articulo/195,https://www.respublica.cl/img/uploads/,"16 de Octubre, 2019",2024-11-29 10:10:08
186,IRP organizó una reflexión pública para aborda...,https://www.respublica.cl/articulo/196,https://www.respublica.cl/img/uploads/,"29 de Octubre, 2019",2024-11-29 10:10:08
187,Presidente de Evópoli Hernán Larraín “El gran ...,https://www.respublica.cl/articulo/197,https://www.respublica.cl/img/uploads/,"05 de Agosto, 2019",2024-11-29 10:10:08
188,A. San Francisco: “Los principales desafíos de...,https://www.respublica.cl/articulo/198,https://www.respublica.cl/img/uploads/,"04 de Abril, 2019",2024-11-29 10:10:08


Radio y tv

In [17]:
base_url = "https://www.respublica.cl/sections/radio-y-television/"
rtv = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=1, headless=False)
rtv

Extrayendo artículos de: https://www.respublica.cl/sections/radio-y-television/1/


,Título,Enlace,URL Imagen,Fecha,Fecha Consulta
0,,https://www.respublica.cl/articulo/1028,https://www.respublica.cl/img/uploads/,,2024-11-29 10:12:32
1,Ricardo Hernández: La seguridad se la llevó el...,https://www.respublica.cl/articulo/775,https://www.respublica.cl/img/uploads/c6e3026b...,"31 de Marzo, 2023",2024-11-29 10:12:32
2,Iriarte:“Putin propone una nueva forma de auto...,https://www.respublica.cl/articulo/1035,https://www.respublica.cl/img/uploads/4920f137...,"19 de Marzo, 2024",2024-11-29 10:12:33
3,"José Francisco Lagos: "" Farmacias Populares y ...",https://www.respublica.cl/articulo/1048,https://www.respublica.cl/img/uploads/8f427edb...,,2024-11-29 10:12:33
4,Emol| Ejecutar convenio policial y colaborar e...,https://www.respublica.cl/articulo/1056,https://www.respublica.cl/img/uploads/fa1b47b6...,"17 de Abril, 2024",2024-11-29 10:12:33
...,...,...,...,...,...
61,"A. Iriarte: ""Las declaraciones de Lavín reflej...",https://www.respublica.cl/articulo/228,https://www.respublica.cl/img/uploads/3d8f7b66...,"26 de Agosto, 2020",2024-11-29 10:12:41
62,"A. Iriarte: ""Lavín podría haber ocupado cualqu...",https://www.respublica.cl/articulo/229,https://www.respublica.cl/img/uploads/304f7f56...,"26 de Agosto, 2020",2024-11-29 10:12:41
63,"Iriarte: ""No escuché a los mismos que rasgan l...",https://www.respublica.cl/articulo/232,https://www.respublica.cl/img/uploads/34c98758...,"02 de Septiembre, 2020",2024-11-29 10:12:41
64,"Iriarte: ""Lo que más necesitamos para poder de...",https://www.respublica.cl/articulo/236,https://www.respublica.cl/img/uploads/023c861a...,"09 de Septiembre, 2020",2024-11-29 10:12:41


Opinión

In [18]:
base_url = "https://www.respublica.cl/sections/opinion/"
op = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=1, headless=False)
op

Extrayendo artículos de: https://www.respublica.cl/sections/opinion/1/


,Título,Enlace,URL Imagen,Fecha,Fecha Consulta
0,"Balance de gobierno: éxitos ideológicos, pelig...",https://www.respublica.cl/articulo/1031,https://www.respublica.cl/img/uploads/2d8ec7ae...,"11 de Marzo, 2024 /",2024-11-29 10:13:26
1,"Julio Borges: ""Sería de vida o muerte que la U...",https://www.respublica.cl/articulo/1038,https://www.respublica.cl/img/uploads/ed64658b...,"22 de Marzo, 2024 /",2024-11-29 10:13:26
2,Argentina está de vuelta,https://www.respublica.cl/articulo/1055,https://www.respublica.cl/img/uploads/dc5f29bf...,"17 de Abril, 2024 / Diario Financiero",2024-11-29 10:13:26
3,"Alejandro San Francisco: ""Sergio de Castro (19...",https://www.respublica.cl/articulo/1062,https://www.respublica.cl/img/uploads/232588ef...,"06 de Mayo, 2024 / El Libero",2024-11-29 10:13:27
4,"Alejandro San Francisco: ""A plena luz del día""",https://www.respublica.cl/articulo/1063,https://www.respublica.cl/img/uploads/b12bfc6d...,/ El Libero,2024-11-29 10:13:27
...,...,...,...,...,...
397,Europa para el siglo XXI,https://www.respublica.cl/articulo/217,https://www.respublica.cl/img/uploads/75db6293...,"02 de Febrero, 2020 / Líbero",2024-11-29 10:14:18
398,Alejandro San Francisco España: repensar el añ...,https://www.respublica.cl/articulo/230,https://www.respublica.cl/img/uploads/27ca88ec...,"30 de Agosto, 2020 / El Imparcial de España",2024-11-29 10:14:18
399,Alejandro San Francisco Paro de camioneros: pr...,https://www.respublica.cl/articulo/231,https://www.respublica.cl/img/uploads/0e6d3a09...,"30 de Agosto, 2020 / Líbero",2024-11-29 10:14:18
400,Alejandro San Francisco: La muerte espiritual ...,https://www.respublica.cl/articulo/235,https://www.respublica.cl/img/uploads/1bce9d3c...,"06 de Septiembre, 2020 / Líbero",2024-11-29 10:14:18


Actualidad

In [19]:
base_url = "https://www.respublica.cl/sections/analisis-de-actualidad/"
act = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=1, headless=False)
act

Extrayendo artículos de: https://www.respublica.cl/sections/analisis-de-actualidad/1/


,Título,Enlace,URL Imagen,Fecha,Fecha Consulta
0,Elecciones Europeas 2024,https://www.respublica.cl/articulo-analisis/1085,https://www.respublica.cl/img/uploads/65584d02...,"27 de Junio, 2024",2024-11-29 10:14:32
1,A 2 AÑOS DEL RECHAZO ¿Cuáles fueron los motivo...,https://www.respublica.cl/articulo-analisis/1105,https://www.respublica.cl/img/uploads/5811352e...,"04 de Septiembre, 2024",2024-11-29 10:14:32
2,PROYECTO DE LEY QUE REFORMA EL SISTEMA DE PENS...,https://www.respublica.cl/articulo-analisis/1109,https://www.respublica.cl/img/uploads/e3f3ed04...,"10 de Septiembre, 2024",2024-11-29 10:14:32
3,ALERTA TEMPRANA Nº6: CASO HERMOSILLA,https://www.respublica.cl/articulo-analisis/1044,https://www.respublica.cl/img/uploads/69219630...,"02 de Abril, 2024",2024-11-29 10:14:32
4,"Balance de gobierno: éxitos ideológicos, pelig...",https://www.respublica.cl/articulo-analisis/1030,https://www.respublica.cl/img/uploads/f99e7f6f...,"11 de Marzo, 2024",2024-11-29 10:14:32
5,Panorama Discusión Constitucional,https://www.respublica.cl/articulo-analisis/698,https://www.respublica.cl/img/uploads/6747680e...,"24 de Octubre, 2022",2024-11-29 10:14:32
6,Resumen de ChatGPT de la propuesta de nueva Co...,https://www.respublica.cl/articulo-analisis/998,https://www.respublica.cl/img/uploads/9001b001...,,2024-11-29 10:14:32
7,Chile el país con el segundo peor sistema trib...,https://www.respublica.cl/articulo-analisis/301,https://www.respublica.cl/img/uploads/c50bc19e...,"24 de Octubre, 2020",2024-11-29 10:14:33
8,INFORME Nº 1 ELECCIÓN PRESIDENCIAL ESTADOS UNIDOS,https://www.respublica.cl/articulo-analisis/298,https://www.respublica.cl/img/uploads/5bd9ffdf...,"06 de Octubre, 2020",2024-11-29 10:14:33
9,Proyectos de ley declarados inadmisibles: gobi...,https://www.respublica.cl/articulo-analisis/211,https://www.respublica.cl/img/uploads/1f14ae7d...,"26 de Junio, 2020",2024-11-29 10:14:33


## Consolidación misma estructura

In [20]:
import pandas as pd

# Lista de DataFrames
dataframes = [notic, rtv, op, act]

# Concatenar los DataFrames
df_concatenado = pd.concat(dataframes, ignore_index=True)

# Ver el DataFrame concatenado
df_concatenado

,Título,Enlace,URL Imagen,Fecha,Fecha Consulta
0,ALERTA TEMPRANA N°46: NUEVO PARO DE FUNCIONARI...,https://www.respublica.cl/articulo/1155,https://www.respublica.cl/img/uploads/36183741...,"28 de Noviembre, 2024",2024-11-29 10:09:46
1,ALERTA TEMPRANA N°45 NUEVO ESCENARIO POLÍTICO ...,https://www.respublica.cl/articulo/1151,https://www.respublica.cl/img/uploads/0d6e962e...,"25 de Noviembre, 2024",2024-11-29 10:09:46
2,ALERTA TEMPRANA N°44 USO DE RECURSOS FISCALES ...,https://www.respublica.cl/articulo/1149,https://www.respublica.cl/img/uploads/3ec61926...,"21 de Noviembre, 2024",2024-11-29 10:09:46
3,ALERTA TEMPRANA N°43 LAS INCOHERENCIAS DEL GOB...,https://www.respublica.cl/articulo/1150,https://www.respublica.cl/img/uploads/8f6b03c8...,"21 de Noviembre, 2024",2024-11-29 10:09:47
4,ALERTA TEMPRANA N°42 ¿ES LEGAL UN PARO DE FUNC...,https://www.respublica.cl/articulo/1146,https://www.respublica.cl/img/uploads/f510a91f...,"19 de Noviembre, 2024",2024-11-29 10:09:47
...,...,...,...,...,...
670,Pensiones en el Blanco,https://www.respublica.cl/articulo-analisis/220,https://www.respublica.cl/img/uploads/a9e54846...,"01 de Agosto, 2020",2024-11-29 10:14:34
671,Migración en Chile: Necesidad de una nueva ley,https://www.respublica.cl/articulo-analisis/225,https://www.respublica.cl/img/uploads/a3d41ec5...,"28 de Agosto, 2020",2024-11-29 10:14:34
672,Reflexiones sobre el plebiscito,https://www.respublica.cl/articulo-analisis/241,https://www.respublica.cl/img/uploads/f8de70fb...,"17 de Septiembre, 2020",2024-11-29 10:14:34
673,Letalidad del COVID-19 y Pobreza Multidimensio...,https://www.respublica.cl/articulo-analisis/242,https://www.respublica.cl/img/uploads/c82b8dfd...,"07 de Octubre, 2020",2024-11-29 10:14:34


## Iteración sobre links

In [ ]:
from selenium.webdriver.common.by import By

def extract_info_from_url(driver, url):
    """
    Extrae información de un artículo dado su URL.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): La URL del artículo del cual extraer la información.
    
    Returns:
        dict: Un diccionario con la información del artículo extraído.
    """
    driver.get(url)
    
    # Esperar a que cargue el contenido (ajustar según la velocidad de carga)
    driver.implicitly_wait(1)
    
    # Intentar obtener la información:
    
    # Título del artículo
    try:
        title = driver.find_element(By.CSS_SELECTOR, "h4.title-estudio").text.strip()
    except:
        title = "No disponible"
    
    # Programa (si está presente)
    try:
        program = driver.find_element(By.CSS_SELECTOR, "p.text-center em").text.strip()
    except:
        program = "No disponible"
    
    # Fecha de publicación
    try:
        publication_date = driver.find_element(By.CSS_SELECTOR, "p.text-right em").text.strip()
    except:
        publication_date = "No disponible"
    
    # Extraer el contenido principal
    try:
        content = driver.find_element(By.CSS_SELECTOR, "div.item-text").text.strip()
    except:
        content = "No disponible"
    
    # Extracto de la introducción (generalmente la primer parte del contenido)
    try:
        introduction = driver.find_element(By.CSS_SELECTOR, "p em").text.strip()
    except:
        introduction = "No disponible"
    
    # Crear el diccionario con los datos
    article_data = {
        "Título": title,
        "Programa": program,
        "Fecha de Publicación": publication_date,
        "Introducción": introduction,
        "Contenido": content,
        "Enlace": url,
    }
    
    return article_data

In [54]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
from tqdm import tqdm  # Importar tqdm para la barra de carga

def process_all_links(jg_df):
    """
    Procesa todos los enlaces de un DataFrame 'jg_df', extrayendo la información de cada artículo
    y almacenándola en un nuevo DataFrame 'publicaciones_jg'.

    Args:
        jg_df (pd.DataFrame): DataFrame que contiene una columna 'Enlace' con los enlaces de los artículos.

    Returns:
        pd.DataFrame: DataFrame con la información extraída de todos los enlaces.
    """
    # Configurar las opciones de Chrome para el modo headless
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Activar modo headless (sin interfaz gráfica)
    chrome_options.add_argument("--disable-gpu")  # Deshabilitar la aceleración por hardware (útil en headless)
    chrome_options.add_argument("--no-sandbox")  # Puede ser necesario en ciertos entornos (como contenedores)

    # Abrir el webdriver con las opciones configuradas
    driver = webdriver.Chrome(options=chrome_options)

    # Crear una lista para almacenar los datos de los artículos
    publicaciones_jg = []

    try:
        # Usar tqdm para mostrar la barra de carga mientras se procesan los enlaces
        for _, row in tqdm(jg_df.iterrows(), total=jg_df.shape[0], desc="Extrayendo datos de artículos"):
            url = row['Enlace']
            # Extraer la información del artículo
            article_data = extract_info_from_url(driver, url)
            
            if article_data:
                publicaciones_jg.append(article_data)
        
        # Crear un DataFrame con los artículos extraídos
        publicaciones_jg_df = pd.DataFrame(publicaciones_jg)
        
        return publicaciones_jg_df
    
    except Exception as e:
        print(f"Error en el proceso: {str(e)}")
        return pd.DataFrame()
    
    finally:
        driver.quit()  # Cerrar el webdriver al finalizar


In [55]:
aver = df_concatenado[:3]
averinfo = process_all_links(aver)
averinfo

Extrayendo datos de artículos: 100%|██████████| 3/3 [00:11<00:00,  3.90s/it]


,Título,Programa,Fecha de Publicación,Introducción,Contenido,Enlace
0,ALERTA TEMPRANA N°46: NUEVO PARO DE FUNCIONARI...,Programa Política y Sociedad,28/11/2024,Programa Política y Sociedad,Programa Política y Sociedad\n28/11/2024\n1. R...,https://www.respublica.cl/articulo/1155
1,ALERTA TEMPRANA N°45 NUEVO ESCENARIO POLÍTICO ...,Programa Política y Sociedad,25/11/2024,Programa Política y Sociedad,Programa Política y Sociedad\n25/11/2024\n1. A...,https://www.respublica.cl/articulo/1151
2,ALERTA TEMPRANA N°44 USO DE RECURSOS FISCALES ...,No disponible,No disponible,No disponible,21/11/2024\nINTRODUCCIÓN\nA propósito del caso...,https://www.respublica.cl/articulo/1149


Para todos pero dara error para varias filas por fecha y programa, correr de nuevo con esa estructura de error.

In [56]:
df_info = process_all_links(df_concatenado)
df_info

Extrayendo datos de artículos: 100%|██████████| 675/675 [1:35:04<00:00,  8.45s/it]


,Título,Programa,Fecha de Publicación,Introducción,Contenido,Enlace
0,ALERTA TEMPRANA N°46: NUEVO PARO DE FUNCIONARI...,Programa Política y Sociedad,28/11/2024,Programa Política y Sociedad,Programa Política y Sociedad\n28/11/2024\n1. R...,https://www.respublica.cl/articulo/1155
1,ALERTA TEMPRANA N°45 NUEVO ESCENARIO POLÍTICO ...,Programa Política y Sociedad,25/11/2024,Programa Política y Sociedad,Programa Política y Sociedad\n25/11/2024\n1. A...,https://www.respublica.cl/articulo/1151
2,ALERTA TEMPRANA N°44 USO DE RECURSOS FISCALES ...,No disponible,No disponible,No disponible,21/11/2024\nINTRODUCCIÓN\nA propósito del caso...,https://www.respublica.cl/articulo/1149
3,ALERTA TEMPRANA N°43 LAS INCOHERENCIAS DEL GOB...,No disponible,No disponible,,Programa Política y Sociedad\n20/11/2024\n1. A...,https://www.respublica.cl/articulo/1150
4,ALERTA TEMPRANA N°42 ¿ES LEGAL UN PARO DE FUNC...,No disponible,No disponible,Programa Política y Sociedad.,Programa Política y Sociedad. ...,https://www.respublica.cl/articulo/1146
...,...,...,...,...,...,...
670,Pensiones en el Blanco,No disponible,No disponible,No disponible,,https://www.respublica.cl/articulo-analisis/220
671,Migración en Chile: Necesidad de una nueva ley,No disponible,No disponible,No disponible,"Chile, dado su mejor nivel de calidad de vida ...",https://www.respublica.cl/articulo-analisis/225
672,Reflexiones sobre el plebiscito,No disponible,No disponible,No disponible,A poco más de un mes de la fecha en la que se ...,https://www.respublica.cl/articulo-analisis/241
673,Letalidad del COVID-19 y Pobreza Multidimensio...,No disponible,No disponible,Interferencia.cl,Durante las últimas semanas se ha planteado en...,https://www.respublica.cl/articulo-analisis/242


In [58]:
df_concatenado

,Título,Enlace,URL Imagen,Fecha,Fecha Consulta
0,ALERTA TEMPRANA N°46: NUEVO PARO DE FUNCIONARI...,https://www.respublica.cl/articulo/1155,https://www.respublica.cl/img/uploads/36183741...,"28 de Noviembre, 2024",2024-11-29 10:09:46
1,ALERTA TEMPRANA N°45 NUEVO ESCENARIO POLÍTICO ...,https://www.respublica.cl/articulo/1151,https://www.respublica.cl/img/uploads/0d6e962e...,"25 de Noviembre, 2024",2024-11-29 10:09:46
2,ALERTA TEMPRANA N°44 USO DE RECURSOS FISCALES ...,https://www.respublica.cl/articulo/1149,https://www.respublica.cl/img/uploads/3ec61926...,"21 de Noviembre, 2024",2024-11-29 10:09:46
3,ALERTA TEMPRANA N°43 LAS INCOHERENCIAS DEL GOB...,https://www.respublica.cl/articulo/1150,https://www.respublica.cl/img/uploads/8f6b03c8...,"21 de Noviembre, 2024",2024-11-29 10:09:47
4,ALERTA TEMPRANA N°42 ¿ES LEGAL UN PARO DE FUNC...,https://www.respublica.cl/articulo/1146,https://www.respublica.cl/img/uploads/f510a91f...,"19 de Noviembre, 2024",2024-11-29 10:09:47
...,...,...,...,...,...
670,Pensiones en el Blanco,https://www.respublica.cl/articulo-analisis/220,https://www.respublica.cl/img/uploads/a9e54846...,"01 de Agosto, 2020",2024-11-29 10:14:34
671,Migración en Chile: Necesidad de una nueva ley,https://www.respublica.cl/articulo-analisis/225,https://www.respublica.cl/img/uploads/a3d41ec5...,"28 de Agosto, 2020",2024-11-29 10:14:34
672,Reflexiones sobre el plebiscito,https://www.respublica.cl/articulo-analisis/241,https://www.respublica.cl/img/uploads/f8de70fb...,"17 de Septiembre, 2020",2024-11-29 10:14:34
673,Letalidad del COVID-19 y Pobreza Multidimensio...,https://www.respublica.cl/articulo-analisis/242,https://www.respublica.cl/img/uploads/c82b8dfd...,"07 de Octubre, 2020",2024-11-29 10:14:34


In [57]:
df_info

,Título,Programa,Fecha de Publicación,Introducción,Contenido,Enlace
0,ALERTA TEMPRANA N°46: NUEVO PARO DE FUNCIONARI...,Programa Política y Sociedad,28/11/2024,Programa Política y Sociedad,Programa Política y Sociedad\n28/11/2024\n1. R...,https://www.respublica.cl/articulo/1155
1,ALERTA TEMPRANA N°45 NUEVO ESCENARIO POLÍTICO ...,Programa Política y Sociedad,25/11/2024,Programa Política y Sociedad,Programa Política y Sociedad\n25/11/2024\n1. A...,https://www.respublica.cl/articulo/1151
2,ALERTA TEMPRANA N°44 USO DE RECURSOS FISCALES ...,No disponible,No disponible,No disponible,21/11/2024\nINTRODUCCIÓN\nA propósito del caso...,https://www.respublica.cl/articulo/1149
3,ALERTA TEMPRANA N°43 LAS INCOHERENCIAS DEL GOB...,No disponible,No disponible,,Programa Política y Sociedad\n20/11/2024\n1. A...,https://www.respublica.cl/articulo/1150
4,ALERTA TEMPRANA N°42 ¿ES LEGAL UN PARO DE FUNC...,No disponible,No disponible,Programa Política y Sociedad.,Programa Política y Sociedad. ...,https://www.respublica.cl/articulo/1146
...,...,...,...,...,...,...
670,Pensiones en el Blanco,No disponible,No disponible,No disponible,,https://www.respublica.cl/articulo-analisis/220
671,Migración en Chile: Necesidad de una nueva ley,No disponible,No disponible,No disponible,"Chile, dado su mejor nivel de calidad de vida ...",https://www.respublica.cl/articulo-analisis/225
672,Reflexiones sobre el plebiscito,No disponible,No disponible,No disponible,A poco más de un mes de la fecha en la que se ...,https://www.respublica.cl/articulo-analisis/241
673,Letalidad del COVID-19 y Pobreza Multidimensio...,No disponible,No disponible,Interferencia.cl,Durante las últimas semanas se ha planteado en...,https://www.respublica.cl/articulo-analisis/242


In [59]:
# Identificar las columnas que están en df_concatenado pero no en df_info
columnas_extra = [col for col in df_concatenado.columns if col not in df_info.columns]

# Seleccionar solo las columnas extra de df_concatenado
df_concatenado_extra = df_concatenado[['Enlace'] + columnas_extra]

# Hacer el merge entre df_info y df_concatenado_extra en la columna "Enlace"
df_combinado = pd.merge(df_info, df_concatenado_extra, on='Enlace', how='left')

# Ver el DataFrame combinado
df_combinado

,Título,Programa,Fecha de Publicación,Introducción,Contenido,Enlace,URL Imagen,Fecha,Fecha Consulta
0,ALERTA TEMPRANA N°46: NUEVO PARO DE FUNCIONARI...,Programa Política y Sociedad,28/11/2024,Programa Política y Sociedad,Programa Política y Sociedad\n28/11/2024\n1. R...,https://www.respublica.cl/articulo/1155,https://www.respublica.cl/img/uploads/36183741...,"28 de Noviembre, 2024",2024-11-29 10:09:46
1,ALERTA TEMPRANA N°45 NUEVO ESCENARIO POLÍTICO ...,Programa Política y Sociedad,25/11/2024,Programa Política y Sociedad,Programa Política y Sociedad\n25/11/2024\n1. A...,https://www.respublica.cl/articulo/1151,https://www.respublica.cl/img/uploads/0d6e962e...,"25 de Noviembre, 2024",2024-11-29 10:09:46
2,ALERTA TEMPRANA N°44 USO DE RECURSOS FISCALES ...,No disponible,No disponible,No disponible,21/11/2024\nINTRODUCCIÓN\nA propósito del caso...,https://www.respublica.cl/articulo/1149,https://www.respublica.cl/img/uploads/3ec61926...,"21 de Noviembre, 2024",2024-11-29 10:09:46
3,ALERTA TEMPRANA N°43 LAS INCOHERENCIAS DEL GOB...,No disponible,No disponible,,Programa Política y Sociedad\n20/11/2024\n1. A...,https://www.respublica.cl/articulo/1150,https://www.respublica.cl/img/uploads/8f6b03c8...,"21 de Noviembre, 2024",2024-11-29 10:09:47
4,ALERTA TEMPRANA N°42 ¿ES LEGAL UN PARO DE FUNC...,No disponible,No disponible,Programa Política y Sociedad.,Programa Política y Sociedad. ...,https://www.respublica.cl/articulo/1146,https://www.respublica.cl/img/uploads/f510a91f...,"19 de Noviembre, 2024",2024-11-29 10:09:47
...,...,...,...,...,...,...,...,...,...
670,Pensiones en el Blanco,No disponible,No disponible,No disponible,,https://www.respublica.cl/articulo-analisis/220,https://www.respublica.cl/img/uploads/a9e54846...,"01 de Agosto, 2020",2024-11-29 10:14:34
671,Migración en Chile: Necesidad de una nueva ley,No disponible,No disponible,No disponible,"Chile, dado su mejor nivel de calidad de vida ...",https://www.respublica.cl/articulo-analisis/225,https://www.respublica.cl/img/uploads/a3d41ec5...,"28 de Agosto, 2020",2024-11-29 10:14:34
672,Reflexiones sobre el plebiscito,No disponible,No disponible,No disponible,A poco más de un mes de la fecha en la que se ...,https://www.respublica.cl/articulo-analisis/241,https://www.respublica.cl/img/uploads/f8de70fb...,"17 de Septiembre, 2020",2024-11-29 10:14:34
673,Letalidad del COVID-19 y Pobreza Multidimensio...,No disponible,No disponible,Interferencia.cl,Durante las últimas semanas se ha planteado en...,https://www.respublica.cl/articulo-analisis/242,https://www.respublica.cl/img/uploads/c82b8dfd...,"07 de Octubre, 2020",2024-11-29 10:14:34


In [60]:
df_combinado.columns

Index(['Título', 'Programa', 'Fecha de Publicación', 'Introducción',
       'Contenido', 'Enlace', 'URL Imagen', 'Fecha', 'Fecha Consulta'],
      dtype='object')

In [62]:
import pandas as pd
import re

# Reemplazar caracteres ilegales en el DataFrame
def clean_illegal_characters(df):
    # Usar una expresión regular para eliminar caracteres de control
    return df.applymap(lambda x: re.sub(r'[\x00-\x1F\x7F]', '', str(x)) if isinstance(x, str) else x)

# Limpiar el DataFrame de caracteres ilegales
df_combinado2 = clean_illegal_characters(df_combinado)

In [63]:
df_final = df_combinado2.copy()
#df_final["Medio de comunicación"] = ""
df_final["Think tank"] = "Res Pública"

df_final  = df_final[['Título', 'Programa', 'Fecha', 'Fecha de Publicación', 'Introducción',
       'Contenido', 'Enlace', 'URL Imagen']]

df_final.to_excel("Res Pública.xlsx", index = False)

In [64]:
df_final

,Título,Programa,Fecha,Fecha de Publicación,Introducción,Contenido,Enlace,URL Imagen
0,ALERTA TEMPRANA N°46: NUEVO PARO DE FUNCIONARI...,Programa Política y Sociedad,"28 de Noviembre, 2024",28/11/2024,Programa Política y Sociedad,Programa Política y Sociedad28/11/20241. RESUM...,https://www.respublica.cl/articulo/1155,https://www.respublica.cl/img/uploads/36183741...
1,ALERTA TEMPRANA N°45 NUEVO ESCENARIO POLÍTICO ...,Programa Política y Sociedad,"25 de Noviembre, 2024",25/11/2024,Programa Política y Sociedad,Programa Política y Sociedad25/11/20241. ANTEC...,https://www.respublica.cl/articulo/1151,https://www.respublica.cl/img/uploads/0d6e962e...
2,ALERTA TEMPRANA N°44 USO DE RECURSOS FISCALES ...,No disponible,"21 de Noviembre, 2024",No disponible,No disponible,21/11/2024INTRODUCCIÓNA propósito del caso Mon...,https://www.respublica.cl/articulo/1149,https://www.respublica.cl/img/uploads/3ec61926...
3,ALERTA TEMPRANA N°43 LAS INCOHERENCIAS DEL GOB...,No disponible,"21 de Noviembre, 2024",No disponible,,Programa Política y Sociedad20/11/20241. ANTEC...,https://www.respublica.cl/articulo/1150,https://www.respublica.cl/img/uploads/8f6b03c8...
4,ALERTA TEMPRANA N°42 ¿ES LEGAL UN PARO DE FUNC...,No disponible,"19 de Noviembre, 2024",No disponible,Programa Política y Sociedad.,Programa Política y Sociedad. ...,https://www.respublica.cl/articulo/1146,https://www.respublica.cl/img/uploads/f510a91f...
...,...,...,...,...,...,...,...,...
670,Pensiones en el Blanco,No disponible,"01 de Agosto, 2020",No disponible,No disponible,,https://www.respublica.cl/articulo-analisis/220,https://www.respublica.cl/img/uploads/a9e54846...
671,Migración en Chile: Necesidad de una nueva ley,No disponible,"28 de Agosto, 2020",No disponible,No disponible,"Chile, dado su mejor nivel de calidad de vida ...",https://www.respublica.cl/articulo-analisis/225,https://www.respublica.cl/img/uploads/a3d41ec5...
672,Reflexiones sobre el plebiscito,No disponible,"17 de Septiembre, 2020",No disponible,No disponible,A poco más de un mes de la fecha en la que se ...,https://www.respublica.cl/articulo-analisis/241,https://www.respublica.cl/img/uploads/f8de70fb...
673,Letalidad del COVID-19 y Pobreza Multidimensio...,No disponible,"07 de Octubre, 2020",No disponible,Interferencia.cl,Durante las últimas semanas se ha planteado en...,https://www.respublica.cl/articulo-analisis/242,https://www.respublica.cl/img/uploads/c82b8dfd...


## Resto de dfs

In [66]:
aver = est[:3]
est_info = process_all_links(est)
est_info

Extrayendo datos de artículos: 100%|██████████| 24/24 [03:43<00:00,  9.31s/it]


,Título,Programa,Fecha de Publicación,Introducción,Contenido,Enlace
0,Nuevo escenario político de gobernadores 2024,No disponible,No disponible,No disponible,El buen resultado de los partidos de oposición...,https://www.respublica.cl/estudio/1152
1,Informe Elecciones CORE y Consejos Municipales...,No disponible,No disponible,No disponible,,https://www.respublica.cl/estudio/1144
2,PRESENTACIÓN: RADIOGRAFÍA AL SENADO 2023,No disponible,No disponible,No disponible,,https://www.respublica.cl/estudio/1024
3,Estudio| Radiografía al Senado 2023,No disponible,No disponible,No disponible,,https://www.respublica.cl/estudio/1025
4,Estudio|Radiografía a la Cámara de Diputados 2023,No disponible,No disponible,No disponible,El informe actual analiza cómo los diputados h...,https://www.respublica.cl/estudio/1051
5,Ranking de Servicios de Salud en Chile Revela ...,Haz clic en la imagen para descargar el estudi...,No disponible,Haz clic en la imagen para descargar el estudi...,"Objetivo del Estudio\nEn un trabajo conjunto, ...",https://www.respublica.cl/estudio/1107
6,ANÁLISIS A LA REFORMA DE PENSIONES,No disponible,No disponible,No disponible,La Reforma de Pensiones del Gobierno se encuen...,https://www.respublica.cl/estudio/1119
7,"RADIOGRAFÍA A LA COMISIÓN EXPERTA, REDACTORA D...",No disponible,No disponible,No disponible,,https://www.respublica.cl/estudio/889
8,RADIOGRAFÍA A LA CÁMARA DE DIPUTADOS DE LA NAC...,No disponible,No disponible,No disponible,,https://www.respublica.cl/estudio/993
9,INFORMACIÓN DEL PLENO DEL CONSEJO CONSTITUCION...,No disponible,No disponible,No disponible,,https://www.respublica.cl/estudio/994


In [67]:
df_concatenado2 = est_info.copy()

In [72]:
est_info.columns

Index(['Título', 'Programa', 'Fecha de Publicación', 'Introducción',
       'Contenido', 'Enlace'],
      dtype='object')

In [73]:
df_final.columns

Index(['Título', 'Programa', 'Fecha', 'Fecha de Publicación', 'Introducción',
       'Contenido', 'Enlace', 'URL Imagen'],
      dtype='object')

In [74]:
import pandas as pd

# Lista de DataFrames
dataframes = [df_combinado2, est_info, edit, eya]

# Concatenar los DataFrames
df_concatenado3 = pd.concat(dataframes, ignore_index=True)

# Ver el DataFrame concatenado
df_concatenado3

,Título,Programa,Fecha de Publicación,Introducción,Contenido,Enlace,URL Imagen,Fecha,Fecha Consulta,Fecha Publicación
0,ALERTA TEMPRANA N°46: NUEVO PARO DE FUNCIONARI...,Programa Política y Sociedad,28/11/2024,Programa Política y Sociedad,Programa Política y Sociedad28/11/20241. RESUM...,https://www.respublica.cl/articulo/1155,https://www.respublica.cl/img/uploads/36183741...,"28 de Noviembre, 2024",2024-11-29 10:09:46,NaN
1,ALERTA TEMPRANA N°45 NUEVO ESCENARIO POLÍTICO ...,Programa Política y Sociedad,25/11/2024,Programa Política y Sociedad,Programa Política y Sociedad25/11/20241. ANTEC...,https://www.respublica.cl/articulo/1151,https://www.respublica.cl/img/uploads/0d6e962e...,"25 de Noviembre, 2024",2024-11-29 10:09:46,NaN
2,ALERTA TEMPRANA N°44 USO DE RECURSOS FISCALES ...,No disponible,No disponible,No disponible,21/11/2024INTRODUCCIÓNA propósito del caso Mon...,https://www.respublica.cl/articulo/1149,https://www.respublica.cl/img/uploads/3ec61926...,"21 de Noviembre, 2024",2024-11-29 10:09:46,NaN
3,ALERTA TEMPRANA N°43 LAS INCOHERENCIAS DEL GOB...,No disponible,No disponible,,Programa Política y Sociedad20/11/20241. ANTEC...,https://www.respublica.cl/articulo/1150,https://www.respublica.cl/img/uploads/8f6b03c8...,"21 de Noviembre, 2024",2024-11-29 10:09:47,NaN
4,ALERTA TEMPRANA N°42 ¿ES LEGAL UN PARO DE FUNC...,No disponible,No disponible,Programa Política y Sociedad.,Programa Política y Sociedad. ...,https://www.respublica.cl/articulo/1146,https://www.respublica.cl/img/uploads/f510a91f...,"19 de Noviembre, 2024",2024-11-29 10:09:47,NaN
...,...,...,...,...,...,...,...,...,...,...
730,INSUMOS CONSTITUCIONALES,NaN,NaN,NaN,NaN,https://www.respublica.cl/ss-ensayos-y-articul...,https://www.respublica.cl/img/uploads/f7861d1e...,NaN,2024-11-29 10:03:35,NaN
731,CONVENCIÓN CONSTITUCIONAL,NaN,NaN,NaN,NaN,https://www.respublica.cl/ss-ensayos-y-articul...,https://www.respublica.cl/img/uploads/c576e39a...,NaN,2024-11-29 10:03:35,NaN
732,PROCESO CONSTITUCIONAL 2023,NaN,NaN,NaN,NaN,https://www.respublica.cl/ss-ensayos-y-articul...,https://www.respublica.cl/img/uploads/b9f18853...,NaN,2024-11-29 10:03:35,NaN
733,DIPLOMADO,NaN,NaN,NaN,NaN,https://www.respublica.cl/ss-ensayos-y-articul...,https://www.respublica.cl/img/uploads/d1c2e82b...,NaN,2024-11-29 10:03:35,NaN


In [75]:
df_fin = df_concatenado3.copy()

In [77]:
df_fin["Think tank"] = "Res Pública"
df_fin.columns

Index(['Título', 'Programa', 'Fecha de Publicación', 'Introducción',
       'Contenido', 'Enlace', 'URL Imagen', 'Fecha', 'Fecha Consulta',
       'Fecha Publicación', 'Think tank'],
      dtype='object')

In [78]:
df_fin["Think tank"] = "Res Pública"

df_fin  = df_fin[['Título', 'Programa', 'Fecha de Publicación', 'Introducción',
       'Contenido', 'Enlace', 'URL Imagen', 'Fecha',
       'Fecha Publicación', 'Think tank']]

df_fin

,Título,Programa,Fecha de Publicación,Introducción,Contenido,Enlace,URL Imagen,Fecha,Fecha Publicación,Think tank
0,ALERTA TEMPRANA N°46: NUEVO PARO DE FUNCIONARI...,Programa Política y Sociedad,28/11/2024,Programa Política y Sociedad,Programa Política y Sociedad28/11/20241. RESUM...,https://www.respublica.cl/articulo/1155,https://www.respublica.cl/img/uploads/36183741...,"28 de Noviembre, 2024",NaN,Res Pública
1,ALERTA TEMPRANA N°45 NUEVO ESCENARIO POLÍTICO ...,Programa Política y Sociedad,25/11/2024,Programa Política y Sociedad,Programa Política y Sociedad25/11/20241. ANTEC...,https://www.respublica.cl/articulo/1151,https://www.respublica.cl/img/uploads/0d6e962e...,"25 de Noviembre, 2024",NaN,Res Pública
2,ALERTA TEMPRANA N°44 USO DE RECURSOS FISCALES ...,No disponible,No disponible,No disponible,21/11/2024INTRODUCCIÓNA propósito del caso Mon...,https://www.respublica.cl/articulo/1149,https://www.respublica.cl/img/uploads/3ec61926...,"21 de Noviembre, 2024",NaN,Res Pública
3,ALERTA TEMPRANA N°43 LAS INCOHERENCIAS DEL GOB...,No disponible,No disponible,,Programa Política y Sociedad20/11/20241. ANTEC...,https://www.respublica.cl/articulo/1150,https://www.respublica.cl/img/uploads/8f6b03c8...,"21 de Noviembre, 2024",NaN,Res Pública
4,ALERTA TEMPRANA N°42 ¿ES LEGAL UN PARO DE FUNC...,No disponible,No disponible,Programa Política y Sociedad.,Programa Política y Sociedad. ...,https://www.respublica.cl/articulo/1146,https://www.respublica.cl/img/uploads/f510a91f...,"19 de Noviembre, 2024",NaN,Res Pública
...,...,...,...,...,...,...,...,...,...,...
730,INSUMOS CONSTITUCIONALES,NaN,NaN,NaN,NaN,https://www.respublica.cl/ss-ensayos-y-articul...,https://www.respublica.cl/img/uploads/f7861d1e...,NaN,NaN,Res Pública
731,CONVENCIÓN CONSTITUCIONAL,NaN,NaN,NaN,NaN,https://www.respublica.cl/ss-ensayos-y-articul...,https://www.respublica.cl/img/uploads/c576e39a...,NaN,NaN,Res Pública
732,PROCESO CONSTITUCIONAL 2023,NaN,NaN,NaN,NaN,https://www.respublica.cl/ss-ensayos-y-articul...,https://www.respublica.cl/img/uploads/b9f18853...,NaN,NaN,Res Pública
733,DIPLOMADO,NaN,NaN,NaN,NaN,https://www.respublica.cl/ss-ensayos-y-articul...,https://www.respublica.cl/img/uploads/d1c2e82b...,NaN,NaN,Res Pública


In [79]:
df_fin.to_excel("Res Pública.xlsx", index = False)

# Consolidación

# Anterior

In [13]:
ip_elm2 = ip_elm[:2]
ip_lyr2 = ip_lyr[:2]
ip_notins2 = ip_notins[:2]

In [14]:
ip_elm_contenido = process_all_links(ip_elm)
ip_elm_contenido

Extrayendo datos de: https://ideapais.cl/noticias/columna-a-donde-estan-que-no-se-ven-por-cristian-stewart-la-segunda/
Extrayendo datos de: https://ideapais.cl/noticias/carta-al-director-el-parche-de-la-confianza-por-arnau-sarra-la-tercera/
Extrayendo datos de: https://ideapais.cl/noticias/columna-los-costos-de-mantener-a-toha-por-cristian-stewart-la-segunda/
Extrayendo datos de: https://ideapais.cl/noticias/columna-volver-a-poner-a-las-personas-en-el-centro-por-matias-domeyko-diario-el-llanquihue/
Extrayendo datos de: https://ideapais.cl/noticias/columna-solidaridad-uc-por-cristian-stewart-la-segunda/
Extrayendo datos de: https://ideapais.cl/noticias/columna-los-catolicos-y-la-conciencia-por-cristian-stewart-la-segunda/
Extrayendo datos de: https://ideapais.cl/noticias/columna-cae-victimas-y-opresores-por-magdalena-vergara-diario-la-tercera/
Extrayendo datos de: https://ideapais.cl/noticias/columna-es-justo-por-juan-pablo-lira-el-dinamo/
Extrayendo datos de: https://ideapais.cl/notici

,Título,Autor,Fecha de Publicación,Categorías,Contenido,Enlace
0,"Columna «A dónde están, que no se ven», por Cr...",IdeaPais,"Nov 21, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",Dentro de las varias aristas que tiene el caso...,https://ideapais.cl/noticias/columna-a-donde-e...
1,"Carta al Director «El parche de la confianza»,...",María José Aravena,"Nov 14, 2024","María José Aravena, Noticias, Noticias Destaca...","Señor Director:\nEn su columna, Miriam Henríqu...",https://ideapais.cl/noticias/carta-al-director...
2,"Columna «Los costos de mantener a Tohá», por C...",IdeaPais,"Nov 7, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",A nadie le gusta que le digan cómo uno debe ha...,https://ideapais.cl/noticias/columna-los-costo...
3,Columna «Volver a poner a las personas en el c...,IdeaPais,"Oct 25, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...","Hace algunos días, en IdeaPaís organizamos un ...",https://ideapais.cl/noticias/columna-volver-a-...
4,"Columna «Solidaridad UC», por Cristián Stewart...",IdeaPais,"Oct 24, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...","Hace 14 años, nació un proyecto político en la...",https://ideapais.cl/noticias/columna-solidarid...
5,"Columna «Los católicos y la conciencia», por C...",IdeaPais,"Sep 26, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",El profesor Carlos Peña inició una importante ...,https://ideapais.cl/noticias/columna-los-catol...
6,"Columna «CAE: víctimas y opresores», por Magda...",IdeaPais,"Sep 18, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",La manera en que el Frente Amplio ha enarbolad...,https://ideapais.cl/noticias/columna-cae-victi...
7,"Columna «¿Es justo?», por Juan Pablo Lira | El...",IdeaPais,"Sep 17, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",Hablemos de la asignación de recursos en educa...,https://ideapais.cl/noticias/columna-es-justo-...
8,"Columna «Conmemoración de hechos importantes»,...",IdeaPais,"Sep 16, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",Sin duda que las primeras semanas de septiembr...,https://ideapais.cl/noticias/columna-conmemora...


In [15]:
ip_notins_contenido = process_all_links(ip_notins)
ip_notins_contenido

Extrayendo datos de: https://ideapais.cl/noticias-institucionales/ideapais-organiza-conversatorio-sobre-desarrollo-urbano-con-alcaldes-de-puerto-montt-y-puerto-varas/
Extrayendo datos de: https://ideapais.cl/noticias-institucionales/candidatos-a-gobernador-regional-de-la-region-de-los-lagos-debatieron-en-el-evento-organizado-por-ideapais/
Extrayendo datos de: https://ideapais.cl/noticias-institucionales/ideapais-invita-a-postular-al-ii-congreso-de-pensamiento-politicas-publicas-profamilia/
Extrayendo datos de: https://ideapais.cl/noticias-institucionales/ideapais-inaugura-sede-regional-en-los-lagos-es-la-cuarta-a-nivel-nacional/
Extrayendo datos de: https://ideapais.cl/sin-categoria/ideapais-realiza-debates-entre-candidatos-a-alcaldes-en-santa-cruz-y-marchigue/
Extrayendo datos de: https://ideapais.cl/noticias-institucionales/con-presencia-de-evelyn-matthei-el-centro-de-estudios-ideapais-realizo-el-congreso-de-politicas-publicas-profamilia/
Extrayendo datos de: https://ideapais.cl/noti

,Título,Autor,Fecha de Publicación,Categorías,Contenido,Enlace
0,IdeaPaís organiza conversatorio sobre desarrol...,María José Aravena,"Nov 28, 2024","María José Aravena, Noticias Destacadas Inicio...",Rodrigo Wainraihgt y Tomás Garate dialogaron j...,https://ideapais.cl/noticias-institucionales/i...
1,Candidatos a Gobernador Regional de la región ...,María José Aravena,"Oct 15, 2024","María José Aravena, Noticias Destacadas Inicio...",El debate se enfocó en temas de seguridad y co...,https://ideapais.cl/noticias-institucionales/c...
2,IdeaPaís invita a postular al II Congreso de P...,IdeaPais,"Jun 7, 2024","IdeaPais, Noticias Institucionales, 0 Comentarios",La instancia se realizará el 27 de septiembre ...,https://ideapais.cl/noticias-institucionales/i...
3,IdeaPaís inaugura sede regional en Los Lagos: ...,IdeaPais,"May 30, 2024","IdeaPais, Noticias IdeaPaís Los Lagos, Noticia...","En el evento, su director regional, Matías Dom...",https://ideapais.cl/noticias-institucionales/i...
4,IdeaPaís realiza debates entre candidatos a al...,María José Aravena,"Oct 17, 2024","María José Aravena, Noticias Destacadas Inicio...",El centro de estudios de inspiración social cr...,https://ideapais.cl/sin-categoria/ideapais-rea...
5,Con presencia de Evelyn Matthei el centro de e...,IdeaPais,"Sep 30, 2024","IdeaPais, Noticias Destacadas Inicio, Noticias...",En el ex Congreso Nacional expusieron ex autor...,https://ideapais.cl/noticias-institucionales/c...
6,La versión 2024 del Encuentro de Líderes Jóven...,IdeaPais,"Jun 6, 2024","IdeaPais, Noticias Institucionales, Noticias O...","IdeaPaís, centro de estudios de inspiración so...",https://ideapais.cl/noticias-ohiggins/la-versi...
7,IdeaPaís y centros de estudios presentaron la ...,IdeaPais,"May 27, 2024","IdeaPais, Noticias Institucionales, 0 Comentarios",Con la presencia de la alcaldesa de Providenci...,https://ideapais.cl/noticias-institucionales/i...
8,Ex autoridades del Biobío entregan sus reflexi...,María José Aravena,"Oct 16, 2024","María José Aravena, Noticias Destacadas Inicio...",El panel de conversación se enfocó en los prin...,https://ideapais.cl/noticias-institucionales/e...
9,«En el Foco» de IdeaPaís analizó las eleccione...,IdeaPais,"Jul 17, 2024","IdeaPais, Noticias Destacadas Inicio, Noticias...",IdeaPaís llevó a cabo el evento online «Europa...,https://ideapais.cl/noticias-institucionales/e...


Crear otra funcion

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def extract_info_from_url(driver, url):
    """
    Extrae información de un artículo dado su URL.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): La URL del artículo del cual extraer la información.
    
    Returns:
        dict: Un diccionario con la información del artículo extraído.
    """
    driver.get(url)
    
    # Esperar a que cargue el contenido (se puede ajustar el tiempo de espera si es necesario)
    driver.implicitly_wait(3)  # Esperar hasta 3 segundos para que se cargue la página
    
    # Extraer la información de interés:
    try:
        # Extraer el título
        title = driver.find_element(By.CSS_SELECTOR, ".et_pb_text_inner h1").text.strip()
    except:
        title = "No disponible"
    
    try:
        # Extraer el año y número de forma específica (de la clase correcta)
        year_issue = driver.find_element(By.CSS_SELECTOR, ".et_pb_text_0 .et_pb_text_inner p").text.strip()
    except:
        year_issue = "No disponible"
    
    try:
        # Extraer todo el contenido del artículo, evitando que incluya el "Año y Número"
        # Asegurarse de que el selector no esté extrayendo elementos no deseados.
        paragraphs = driver.find_elements(By.CSS_SELECTOR, ".et_pb_text_2 .et_pb_text_inner p")
        content = "\n".join([p.text.strip() for p in paragraphs if p.text.strip() != ""])  # Unir todos los párrafos
    except:
        content = "No disponible"
    
    try:
        # Esperar hasta que el enlace de descarga esté presente
        download_link_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[text()='Descarga la revista aquí']"))
        )
        download_link = download_link_element.get_attribute("href")
    except Exception as e:
        print(f"Error al obtener el enlace de descarga: {e}")
        download_link = "No disponible"
    
    # Crear el diccionario con los datos
    article_data = {
        "Título": title,
        "Año y Número": year_issue,
        "Contenido": content,
        "Enlace de Descarga": download_link,
        "Enlace": url,
    }
    
    return article_data

def process_all_links(jg_df, headless=True):
    """
    Procesa todos los enlaces de un DataFrame 'jg_df', extrayendo la información de cada artículo
    y almacenándola en un nuevo DataFrame 'publicaciones_jg'.
    
    Args:
        jg_df (pd.DataFrame): DataFrame que contiene una columna 'Enlace' con los enlaces de los artículos.
        headless (bool): Determina si el modo headless debe estar activado o no. Por defecto es True.
    
    Returns:
        pd.DataFrame: DataFrame con la información extraída de todos los enlaces.
    """
    # Configurar las opciones de Chrome para el modo headless
    chrome_options = Options()
    
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless (sin interfaz gráfica)
    
    chrome_options.add_argument("--disable-gpu")  # Deshabilitar la aceleración por hardware (útil en headless)
    chrome_options.add_argument("--no-sandbox")  # Puede ser necesario en ciertos entornos (como contenedores)

    # Abrir el webdriver con las opciones configuradas
    driver = webdriver.Chrome(options=chrome_options)

    # Crear una lista para almacenar los datos de los artículos
    publicaciones_jg = []

    try:
        for _, row in jg_df.iterrows():
            url = row['Enlace']
            print(f"Extrayendo datos de: {url}")
            
            # Extraer la información del artículo
            article_data = extract_info_from_url(driver, url)
            
            if article_data:
                publicaciones_jg.append(article_data)
        
        # Crear un DataFrame con los artículos extraídos
        publicaciones_jg_df = pd.DataFrame(publicaciones_jg)
        
        return publicaciones_jg_df
    
    except Exception as e:
        print(f"Error en el proceso: {str(e)}")
        return pd.DataFrame()
    
    finally:
        driver.quit()  # Cerrar el webdriver al finalizar

In [17]:
ip_lyr_contenido = process_all_links(ip_lyr, headless=True)
ip_lyr_contenido

Extrayendo datos de: https://ideapais.cl/revista-raices-la-familia-un-bien-publico-vi-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-dilemas-de-la-juventud-v-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-pensar-una-centro-derecha-reformista-iv-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-reformismo-y-constitucion-iii-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-desafios-constitucionales-ii-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-solidaridad-i-edicion/
Extrayendo datos de: https://ideapais.cl/producto/gonzalo-vial-politica-y-crisis-social-2da-edicion-ampliada/
Error al obtener el enlace de descarga: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6A7556CB5+28821]
	(No symbol) [0x00007FF6A74C3840]
	(No symbol) [0x00007FF6A736578A]
	(No symbol) [0x00007FF6A73B91BE]
	(No symbol) [0x00007FF6A73B94AC]
	(No symbol) [0x00007FF6A7402647]
	(No symbol) [0x00007FF6A73DF33F]
	(No symbol) [0x0000

,Título,Año y Número,Contenido,Enlace de Descarga,Enlace
0,Revista Raíces «La familia: un bien público»,"AÑO 2024, Nº6",El motor del trabajo de IdeaPaís durante estos...,https://ideapais.cl/wp-content/uploads/2024/04...,https://ideapais.cl/revista-raices-la-familia-...
1,Revista Raíces «Dilemas de la juventud»,"AÑO 2023, Nº5",La quinta edición de Raíces está dedicada a re...,https://ideapais.cl/wp-content/uploads/2023/08...,https://ideapais.cl/revista-raices-dilemas-de-...
2,Revista Raíces «Pensar una centro derecha refo...,"AÑO 2022, Nº4","Este número de la revista Raíces, busca dar un...",https://ideapais.cl/wp-content/uploads/2023/02...,https://ideapais.cl/revista-raices-pensar-una-...
3,Revista Raíces «Reformismo y Constitución»,"AÑO 2022, Nº3",La tercera edición de la revista Raíces nos en...,https://ideapais.cl/wp-content/uploads/2022/08...,https://ideapais.cl/revista-raices-reformismo-...
4,Revista Raíces «Desafíos constitucionales»,"AÑO 2021, Nº2",La experiencia de los últimos años nos ha ense...,https://ideapais.cl/wp-content/uploads/2022/08...,https://ideapais.cl/revista-raices-desafios-co...
5,Revista Raíces «Solidaridad»,"AÑO 2020, Nº1","A diez años del nacimiento de IdeaPaís, presen...",https://ideapais.cl/wp-content/uploads/2022/08...,https://ideapais.cl/revista-raices-solidaridad...
6,No disponible,No disponible,,No disponible,https://ideapais.cl/producto/gonzalo-vial-poli...
7,No disponible,No disponible,,No disponible,https://ideapais.cl/producto/cuestiones-obreras/
8,No disponible,No disponible,,No disponible,https://ideapais.cl/producto/constitucion-soli...
9,No disponible,No disponible,,No disponible,https://ideapais.cl/producto/por-que-ha-fracas...


## Consolidar archivos

In [18]:
import pandas as pd

# Lista de DataFrames
dataframes = [ip_pp, ip_ml, ip_dc, ip_elm_contenido, ip_lyr_contenido, ip_notins_contenido, ip_inic]

# Concatenar los DataFrames
df_concatenado = pd.concat(dataframes, ignore_index=True)

# Ver el DataFrame concatenado
df_concatenado

,Título,Enlace,Fecha Publicación,URL Imagen,Contenido,Fecha Consulta,Autor,Fecha de Publicación,Categorías,Año y Número,Enlace de Descarga
0,Reporte II Congreso de Políticas Públicas Prof...,https://ideapais.cl/wp-content/uploads/2024/10...,"1 de Oct, 2024",https://ideapais.cl/wp-content/uploads/2024/10...,,2024-11-29 01:29:43,NaN,NaN,NaN,NaN,NaN
1,Presentación: Labores de cuidados en la encues...,https://ideapais.cl/wp-content/uploads/2024/09...,"3 de Sep, 2024",https://ideapais.cl/wp-content/uploads/2024/09...,,2024-11-29 01:29:43,NaN,NaN,NaN,NaN,NaN
2,Chile desde las comunas: Propuestas locales pa...,https://ideapais.cl/wp-content/uploads/2024/08...,"28 de Ago, 2024",https://ideapais.cl/wp-content/uploads/2024/08...,,2024-11-29 01:29:43,NaN,NaN,NaN,NaN,NaN
3,Radiografía: Zoom a la clase media chilena,https://ideapais.cl/wp-content/uploads/2024/07...,"18 de Jul, 2024",https://ideapais.cl/wp-content/uploads/2024/07...,,2024-11-29 01:29:43,NaN,NaN,NaN,NaN,NaN
4,Radiografía: Matrícula extranjera en el sistem...,https://ideapais.cl/wp-content/uploads/2024/07...,"5 de Jul, 2024",https://ideapais.cl/wp-content/uploads/2024/07...,,2024-11-29 01:29:43,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
97,Memoria 2020,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33,NaN,NaN,NaN,NaN,NaN
98,Memoria 2022,https://indd.adobe.com/view/2cb08e30-369b-439e...,"Feb 16, 2024","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33,NaN,NaN,NaN,NaN,NaN
99,Memoria 2019,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33,NaN,NaN,NaN,NaN,NaN
100,Memoria 2021,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33,NaN,NaN,NaN,NaN,NaN


In [19]:
df_final = df_concatenado.copy()

In [ ]:
#df_final["Medio de comunicación"] = ""
df_final["Think tank"] = "Idea País"

df_final  = df_final[['Título', 'Enlace', 'Fecha Publicación', 'URL Imagen', 'Contenido', 'Autor', 'Fecha de Publicación', 'Categorías',
       'Año y Número', 'Enlace de Descarga']]

df_final.to_excel("Idea País.xlsx", index = False)

## Guardar archivos

In [22]:
df_final.to_excel("Idea País.xlsx", index = False)